In [4]:
# install all requirements via pip
!pip install torch transformers pandas datasets transformers[torch]

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# initialise GPT2 model (importing Hugging Face)

from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [15]:
# test out the base GPT2 model

prompt = "Once upon a time"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(inputs.input_ids, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)

output_string = tokenizer.batch_decode(outputs)

output_string

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Once upon a time of opportunity, I found myself thinking, "Well, I’ve always done this with, and I thought, what about it? If I was a musician who had always been a musician, it might sound interesting to me but… I’ve never been quite so good at it.” I am so honored to say I feel like a musician at all. It’t just me and my community as a whole, but this particular part of me is not just me,']

In [16]:
# import our dataset from hugging face
from datasets import load_dataset

short_stories_dataset = load_dataset("roneneldan/TinyStories")
short_stories_dataset

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})

In [17]:
short_stories_dataset["train"][2]

{'text': 'One day, a little fish named Fin was swimming near the shore. He saw a big crab and wanted to be friends. "Hi, I am Fin. Do you want to play?" asked the little fish. The crab looked at Fin and said, "No, I don\'t want to play. I am cold and I don\'t feel fine."\n\nFin felt sad but wanted to help the crab feel better. He swam away and thought of a plan. He remembered that the sun could make things warm. So, Fin swam to the top of the water and called to the sun, "Please, sun, help my new friend feel fine and not freeze!"\n\nThe sun heard Fin\'s call and shone its warm light on the shore. The crab started to feel better and not so cold. He saw Fin and said, "Thank you, little fish, for making me feel fine. I don\'t feel like I will freeze now. Let\'s play together!" And so, Fin and the crab played and became good friends.'}

In [21]:
# Get a smaller subset of data from the dataset

small_story_dataset = load_dataset("roneneldan/TinyStories", split="train[:1000]")
small_story_dataset = small_story_dataset.train_test_split(train_size=0.8)

small_story_dataset

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 800
    })
    test: Dataset({
        features: ['text'],
        num_rows: 200
    })
})

In [23]:
[len(x["text"].split(" ")) for x in small_story_dataset["train"]]

[220,
 521,
 360,
 150,
 103,
 167,
 205,
 354,
 470,
 148,
 135,
 194,
 186,
 387,
 142,
 120,
 278,
 138,
 99,
 179,
 522,
 165,
 129,
 149,
 144,
 299,
 116,
 229,
 280,
 161,
 189,
 157,
 265,
 151,
 157,
 183,
 241,
 162,
 174,
 138,
 135,
 112,
 195,
 150,
 188,
 155,
 145,
 153,
 166,
 136,
 215,
 168,
 161,
 157,
 119,
 97,
 174,
 216,
 143,
 107,
 265,
 162,
 171,
 129,
 120,
 127,
 396,
 167,
 140,
 112,
 154,
 111,
 125,
 174,
 106,
 194,
 104,
 255,
 117,
 411,
 177,
 142,
 142,
 305,
 181,
 328,
 278,
 171,
 281,
 167,
 173,
 108,
 165,
 149,
 113,
 167,
 151,
 117,
 139,
 140,
 215,
 179,
 137,
 140,
 155,
 187,
 188,
 324,
 115,
 116,
 150,
 104,
 127,
 171,
 197,
 170,
 123,
 215,
 152,
 215,
 185,
 309,
 136,
 159,
 147,
 138,
 102,
 157,
 179,
 179,
 161,
 127,
 154,
 152,
 158,
 94,
 110,
 117,
 133,
 149,
 79,
 276,
 188,
 196,
 110,
 138,
 161,
 176,
 176,
 150,
 146,
 142,
 220,
 318,
 181,
 185,
 141,
 226,
 158,
 328,
 143,
 151,
 150,
 342,
 132,
 176,
 179,
 2

In [24]:
# Tokenize our dataset
def preprocess_batch(batch):
    all_text_items = batch["text"]
    trimmed_text_items = [x[:500] for x in all_text_items]
    return tokenizer(trimmed_text_items)

tokenized_dataset = small_story_dataset.map(
    preprocess_batch, 
    batched=True, 
    batch_size=10, 
    remove_columns=small_story_dataset["train"].column_names
)

tokenized_dataset
    

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 800
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 200
    })
})

In [25]:
tokenized_dataset["train"][0]

{'input_ids': [7454,
  2402,
  257,
  640,
  11,
  612,
  373,
  257,
  1310,
  2576,
  3706,
  3619,
  88,
  13,
  3619,
  88,
  6151,
  2712,
  351,
  607,
  15392,
  13,
  1375,
  8288,
  284,
  6129,
  340,
  1029,
  287,
  262,
  6766,
  290,
  7906,
  1088,
  290,
  1088,
  13,
  220,
  198,
  198,
  3198,
  1110,
  11,
  3619,
  88,
  2227,
  284,
  1011,
  607,
  15392,
  329,
  257,
  6594,
  1088,
  262,
  3952,
  13,
  1375,
  1234,
  319,
  607,
  10012,
  290,
  572,
  673,
  1816,
  13,
  198,
  198,
  2953,
  262,
  3952,
  11,
  3619,
  88,
  373,
  13226,
  7348,
  287,
  262,
  6766,
  618,
  477,
  286,
  257,
  4802,
  11,
  257,
  1263,
  35253,
  286,
  2344,
  1625,
  290,
  607,
  15392,
  11406,
  287,
  257,
  16723,
  13,
  198,
  198,
  14295,
  88,
  373,
  845,
  9247,
  290,
  2067,
  284,
  3960,
  13,
  24975,
  11,
  281,
  10966,
  8223,
  1625,
  7976,
  15816,
  625,
  220],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [28]:
# Create a Data Collator
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
data_collator

DataCollatorForLanguageModeling(tokenizer=GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [29]:
# Create our Trainer and TrainingArguments
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=10
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
    data_collator=data_collator
)

trainer.train()


In [ ]:
# Start training!

In [ ]:
# Inference